### 커피빈 크롤링

라이브러리 import

In [70]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
import time

크롤링을 수행하는 함수 정의

In [71]:
def CoffeeBean_store(result):
    CoffeeBean_URL = "https://www.coffeebeankorea.com/store/store.asp"
    wd = webdriver.Chrome('./chromedriver')

    # for i in range(1, 13):  #매장 수 만큼 반복    
    wd.get(CoffeeBean_URL)
    time.sleep(1)
    
    element = wd.find_element_by_id('region_srh')
    element.click()
    time.sleep(1)

    # dropdown menu를 펼침
    element = wd.find_element(By.ID, 'localTitle')
    element.click()
    time.sleep(1)
    
    element = wd.find_element(By.ID, 'storeLocal')
    elements = element.find_elements(By.TAG_NAME, 'li')     # 지역명이 나와있는 리스트들을 찾고
    for element in elements:        
        if element.text == '서울':  # 텍스트에 서울을 갖고 있는 요소를 찾아냅니다.
            break
    
    element.find_element(By.TAG_NAME, 'a')  # 해당 요소 하위 태그인 a 태그를 찾아내 클릭
    element.click()
    
    time.sleep(1)
    
    # 성남시 분당구 찾기 시작
    # dropdown menu를 펼침
    element = wd.find_element_by_id('localTitle2')
    element.click()
    time.sleep(1)
    
    element = wd.find_element(By.ID, 'storeLocal2')
    elements = element.find_elements(By.TAG_NAME, 'li')
    for element in elements:
        if element.text == '송파구':
            break
    
    element.find_element(By.TAG_NAME, 'a')
    element.click()
    time.sleep(1)  # 목록 띄울 동안 1초 대기

    for i in range(13):
        try:
            elements = wd.find_elements(By.CLASS_NAME, 'btn_style6')
            # wd.execute_script("storePop2(%d)" %i)
            elements[i].click()
            time.sleep(1) #스크립트 실행 할 동안 1초 대기
            html = wd.page_source
            soupCB = BeautifulSoup(html, 'html.parser')
            store_name_h2 = soupCB.select("div.store_txt > h2")
            store_name = store_name_h2[0].string
            print(store_name)  #매장 이름 출력하기
            store_info = soupCB.select("div.store_txt > table.store_table > tbody > tr > td")
            store_address_list = list(store_info[2])
            store_address = store_address_list[0]
            store_phone = store_info[3].string
            result.append([store_name]+[store_address]+[store_phone])
        except:
            continue
            
        element = wd.find_element(By.ID, 'closeBtn1ActiveBtn')
        element.click()
        time.sleep(0.5)
    
    return



함수 호출 및 DataFrame을 csv파일로 저장

In [72]:

def main():
    result = []
    print('CoffeeBean store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>')
    CoffeeBean_store(result)  #[CODE 1]
    
    CB_tbl = pd.DataFrame(result, columns=('store', 'address','phone'))
    CB_tbl.to_csv('./201935256문지환송파커피빈.csv', encoding='cp949', mode='w', index=True)


main()

CoffeeBean store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>


/var/folders/m0/5pqtbll13p97hjn1mqvl1mwh0000gn/T/ipykernel_1014/4160513595.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome('./chromedriver')
/var/folders/m0/5pqtbll13p97hjn1mqvl1mwh0000gn/T/ipykernel_1014/4160513595.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = wd.find_element_by_id('region_srh')
/var/folders/m0/5pqtbll13p97hjn1mqvl1mwh0000gn/T/ipykernel_1014/4160513595.py:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = wd.find_element_by_id('localTitle2')


송파헬리오시티점
잠실향군타워점
문정동환인제약빌딩점
방이역4번출구점
위례2차아이파크점
현대시티몰가든파이브점
몽촌토성역점
문정역엠스테이트점
송파파크하비오점
석촌호수점
올림픽플라자점
방이점
잠실신천점


### 지오코드 API 사용

In [73]:
import pandas as pd

CB = pd.read_csv('./201935256문지환송파커피빈.csv', encoding='CP949', index_col=0, header=0, engine='python')

CB.head()  #작업 내용 확인용 출력

,store,address,phone
0,송파헬리오시티점,서울시 송파구 송파대로 345 ( 헬리오시티상가 3블럭 A동 B1층 22호 ),02-777-3363
1,잠실향군타워점,서울특별시 송파구 올림픽로35길 123 1층,02-412-8812
2,문정동환인제약빌딩점,서울특별시 송파구 법원로6길 11 환인빌딩 1층,02-449-9569
3,방이역4번출구점,서울 송파구 방이동 206-11,02-420-0386
4,위례2차아이파크점,서울시 송파구 위례광장로 230 위례2차 아이파크 B동 1층,02-449-9703


각 주소 전처리

In [74]:
addr = []

for address in CB.address:
    addr.append(str(address).split())

#작업 내용 확인용 출력
print('데이터 개수 : %d' % len(addr)) 
addr  

데이터 개수 : 13


[['서울시', '송파구', '송파대로', '345', '(', '헬리오시티상가', '3블럭', 'A동', 'B1층', '22호', ')'],
 ['서울특별시', '송파구', '올림픽로35길', '123', '1층'],
 ['서울특별시', '송파구', '법원로6길', '11', '환인빌딩', '1층'],
 ['서울', '송파구', '방이동', '206-11'],
 ['서울시', '송파구', '위례광장로', '230', '위례2차', '아이파크', 'B동', '1층'],
 ['서울특별시', '송파구', '충민로', '66', '가든파이브', '라이프동', '테크노관', '1층'],
 ['서울시', '송파구', '방이동', '44-2,', '장은빌딩', '1층'],
 ['서울특별시', '송파구', '법원로', '114,', '엠스테이트', 'B동', '지하', '1층,', 'B145-147'],
 ['서울특별시', '송파구', '송파대로', '111', '송파파크하비오', '205동', '1층', '110-113'],
 ['서울시', '송파구', '석촌호수로', '222', '제이타워', '1층'],
 ['서울시', '송파구', '양재대로', '1222', '올림픽선수A', '1층212~3호'],
 ['서울시', '송파구', '오금로', '11길', '7'],
 ['서울시', '송파구', '석촌호수로', '118', '1층']]

주소 표준화

In [75]:
addr2 = []

# addr에서 행정구역 표준 이름이 아닌것 수정하기
for i in range(len(addr)):
    if addr[i][0] == "서울": addr[i][0]="서울특별시"
    elif addr[i][0] == "서울시": addr[i][0]="서울특별시"
    elif addr[i][0] == "부산시": addr[i][0]="부산광역시"
    elif addr[i][0] == "인천": addr[i][0]="인천광역시"
    elif addr[i][0] == "광주": addr[i][0]="광주광역시"
    elif addr[i][0] == "대전시": addr[i][0]="대전광역시"
    elif addr[i][0] == "울산시": addr[i][0]="울산광역시"    
    elif addr[i][0] == "세종시": addr[i][0]="세종특별자치시"
    elif addr[i][0] == "경기": addr[i][0]="경기도"
    elif addr[i][0] == "충북": addr[i][0]="충청북도"
    elif addr[i][0] == "충남": addr[i][0]="충청남도"
    elif addr[i][0] == "전북": addr[i][0]="전라북도"
    elif addr[i][0] == "전남": addr[i][0]="전라남도"
    elif addr[i][0] == "경북": addr[i][0]="경상북도"
    elif addr[i][0] == "경남": addr[i][0]="경상남도"
    elif addr[i][0] == "제주": addr[i][0]="제주특별자치도"
    elif addr[i][0] == "제주도": addr[i][0]="제주특별자치도"
    elif addr[i][0] == "제주시": addr[i][0]="제주특별자치도"                                
       
    addr2.append(' '.join(addr[i]))  

addr2 #작업 내용 확인용 출력

['서울특별시 송파구 송파대로 345 ( 헬리오시티상가 3블럭 A동 B1층 22호 )',
 '서울특별시 송파구 올림픽로35길 123 1층',
 '서울특별시 송파구 법원로6길 11 환인빌딩 1층',
 '서울특별시 송파구 방이동 206-11',
 '서울특별시 송파구 위례광장로 230 위례2차 아이파크 B동 1층',
 '서울특별시 송파구 충민로 66 가든파이브 라이프동 테크노관 1층',
 '서울특별시 송파구 방이동 44-2, 장은빌딩 1층',
 '서울특별시 송파구 법원로 114, 엠스테이트 B동 지하 1층, B145-147',
 '서울특별시 송파구 송파대로 111 송파파크하비오 205동 1층 110-113',
 '서울특별시 송파구 석촌호수로 222 제이타워 1층',
 '서울특별시 송파구 양재대로 1222 올림픽선수A 1층212~3호',
 '서울특별시 송파구 오금로 11길 7',
 '서울특별시 송파구 석촌호수로 118 1층']

list to DataFrame

In [76]:
addr2 = pd.DataFrame(addr2, columns=['address2'])

addr2 #작업 내용 확인용 출력

,address2
0,서울특별시 송파구 송파대로 345 ( 헬리오시티상가 3블럭 A동 B1층 22호 )
1,서울특별시 송파구 올림픽로35길 123 1층
2,서울특별시 송파구 법원로6길 11 환인빌딩 1층
3,서울특별시 송파구 방이동 206-11
4,서울특별시 송파구 위례광장로 230 위례2차 아이파크 B동 1층
5,서울특별시 송파구 충민로 66 가든파이브 라이프동 테크노관 1층
6,"서울특별시 송파구 방이동 44-2, 장은빌딩 1층"
7,"서울특별시 송파구 법원로 114, 엠스테이트 B동 지하 1층, B145-147"
8,서울특별시 송파구 송파대로 111 송파파크하비오 205동 1층 110-113
9,서울특별시 송파구 석촌호수로 222 제이타워 1층


기존 DataFrame에 병합

In [77]:
CB2 = pd.concat([CB, addr2],  axis=1 )

CB2.head()  #작업 내용 확인용 출력

,store,address,phone,address2
0,송파헬리오시티점,서울시 송파구 송파대로 345 ( 헬리오시티상가 3블럭 A동 B1층 22호 ),02-777-3363,서울특별시 송파구 송파대로 345 ( 헬리오시티상가 3블럭 A동 B1층 22호 )
1,잠실향군타워점,서울특별시 송파구 올림픽로35길 123 1층,02-412-8812,서울특별시 송파구 올림픽로35길 123 1층
2,문정동환인제약빌딩점,서울특별시 송파구 법원로6길 11 환인빌딩 1층,02-449-9569,서울특별시 송파구 법원로6길 11 환인빌딩 1층
3,방이역4번출구점,서울 송파구 방이동 206-11,02-420-0386,서울특별시 송파구 방이동 206-11
4,위례2차아이파크점,서울시 송파구 위례광장로 230 위례2차 아이파크 B동 1층,02-449-9703,서울특별시 송파구 위례광장로 230 위례2차 아이파크 B동 1층


In [78]:
%pip install folium

You should consider upgrading via the '/Users/hoho/hoho_dir/Development/github/2022-1-Bigdata/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


필요한 라이브러리 import

In [79]:
import folium

In [80]:
import requests
from urllib import parse
import pandas as pd
import re

API를 요청하는 함수

In [81]:
def geocode(address, type='ROAD'):
    result = requests.get("http://api.vworld.kr/req/address?service=address"
        + "&request=getcoord"
        + "&address=" + parse.quote(address) + "&type=" + type
        + "&key=838DDA11-A005-360D-8388-51D7EFC76EFC")
    return result

In [82]:
# 경도, 위도 필드 만들기
CB2['경도'] = 0.0
CB2['위도'] = 0.0
CB2

,store,address,phone,address2,경도,위도
0,송파헬리오시티점,서울시 송파구 송파대로 345 ( 헬리오시티상가 3블럭 A동 B1층 22호 ),02-777-3363,서울특별시 송파구 송파대로 345 ( 헬리오시티상가 3블럭 A동 B1층 22호 ),0.0,0.0
1,잠실향군타워점,서울특별시 송파구 올림픽로35길 123 1층,02-412-8812,서울특별시 송파구 올림픽로35길 123 1층,0.0,0.0
2,문정동환인제약빌딩점,서울특별시 송파구 법원로6길 11 환인빌딩 1층,02-449-9569,서울특별시 송파구 법원로6길 11 환인빌딩 1층,0.0,0.0
3,방이역4번출구점,서울 송파구 방이동 206-11,02-420-0386,서울특별시 송파구 방이동 206-11,0.0,0.0
4,위례2차아이파크점,서울시 송파구 위례광장로 230 위례2차 아이파크 B동 1층,02-449-9703,서울특별시 송파구 위례광장로 230 위례2차 아이파크 B동 1층,0.0,0.0
5,현대시티몰가든파이브점,서울특별시 송파구 충민로 66 가든파이브 라이프동 테크노관 1층,02-2673-2110,서울특별시 송파구 충민로 66 가든파이브 라이프동 테크노관 1층,0.0,0.0
6,몽촌토성역점,"서울시 송파구 방이동 44-2, 장은빌딩 1층",02-2141-0407,"서울특별시 송파구 방이동 44-2, 장은빌딩 1층",0.0,0.0
7,문정역엠스테이트점,"서울특별시 송파구 법원로 114, 엠스테이트 B동 지하 1층, B145-147",02-6177-9080,"서울특별시 송파구 법원로 114, 엠스테이트 B동 지하 1층, B145-147",0.0,0.0
8,송파파크하비오점,서울특별시 송파구 송파대로 111 송파파크하비오 205동 1층 110-113,02-449-6881,서울특별시 송파구 송파대로 111 송파파크하비오 205동 1층 110-113,0.0,0.0
9,석촌호수점,서울시 송파구 석촌호수로 222 제이타워 1층,02-415-1235,서울특별시 송파구 석촌호수로 222 제이타워 1층,0.0,0.0


API 호출하고 각 주소에 해당하는 좌표를 수집

In [83]:
for i in range(len(CB2)):
    result = None
    coord_x = coord_y = 0.0
    result = geocode(CB2.at[i, 'address2']).json()
    if result['response']['status'] != 'NOT_FOUND':
        coord_x = result['response']['result']['point']['x']
        coord_y = result['response']['result']['point']['y']
    else:
        # 도로명 주소 공백이 없을 때 못찾는 경우 처리
        str1 = re.sub('([a-zA-Zㄱ-힗]+로)([0-9]+)', '\\1 \\2', CB2.at[i, 'address2'])
        result = geocode(str1).json()
        if result['response']['status'] != 'NOT_FOUND':
            coord_x = result['response']['result']['point']['x']
            coord_y = result['response']['result']['point']['y']
        else:      
            # 도로명 주소로 못찾을 경우 지번 주소로 검색
            result = geocode(CB2.at[i, 'address2'], 'PARCEL').json()
            if result['response']['status'] != 'NOT_FOUND':
                coord_x = result['response']['result']['point']['x']
                coord_y = result['response']['result']['point']['y']
            else: 
                continue
    
    CB2.at[i, '경도'] = round(float(coord_x), 4)
    CB2.at[i, '위도'] = round(float(coord_y), 4)

CB2

,store,address,phone,address2,경도,위도
0,송파헬리오시티점,서울시 송파구 송파대로 345 ( 헬리오시티상가 3블럭 A동 B1층 22호 ),02-777-3363,서울특별시 송파구 송파대로 345 ( 헬리오시티상가 3블럭 A동 B1층 22호 ),127.1025,37.4977
1,잠실향군타워점,서울특별시 송파구 올림픽로35길 123 1층,02-412-8812,서울특별시 송파구 올림픽로35길 123 1층,127.1012,37.5166
2,문정동환인제약빌딩점,서울특별시 송파구 법원로6길 11 환인빌딩 1층,02-449-9569,서울특별시 송파구 법원로6길 11 환인빌딩 1층,127.1222,37.4836
3,방이역4번출구점,서울 송파구 방이동 206-11,02-420-0386,서울특별시 송파구 방이동 206-11,127.1258,37.5094
4,위례2차아이파크점,서울시 송파구 위례광장로 230 위례2차 아이파크 B동 1층,02-449-9703,서울특별시 송파구 위례광장로 230 위례2차 아이파크 B동 1층,127.1438,37.4786
5,현대시티몰가든파이브점,서울특별시 송파구 충민로 66 가든파이브 라이프동 테크노관 1층,02-2673-2110,서울특별시 송파구 충민로 66 가든파이브 라이프동 테크노관 1층,127.1251,37.4777
6,몽촌토성역점,"서울시 송파구 방이동 44-2, 장은빌딩 1층",02-2141-0407,"서울특별시 송파구 방이동 44-2, 장은빌딩 1층",127.1127,37.5169
7,문정역엠스테이트점,"서울특별시 송파구 법원로 114, 엠스테이트 B동 지하 1층, B145-147",02-6177-9080,"서울특별시 송파구 법원로 114, 엠스테이트 B동 지하 1층, B145-147",127.1209,37.4854
8,송파파크하비오점,서울특별시 송파구 송파대로 111 송파파크하비오 205동 1층 110-113,02-449-6881,서울특별시 송파구 송파대로 111 송파파크하비오 205동 1층 110-113,127.1231,37.4812
9,석촌호수점,서울시 송파구 석촌호수로 222 제이타워 1층,02-415-1235,서울특별시 송파구 석촌호수로 222 제이타워 1층,127.1016,37.5070


In [84]:
len(CB2) #확인용 출력

13

In [85]:
map_CB = folium.Map(location=[37.4854, 127.1209], zoom_start = 15)

파란색 깃발 모양의 마커 생성

In [86]:
for i, store in CB2.iterrows():   
    folium.Marker(location=[store['위도'], store['경도']], popup= store['store'], icon=folium.Icon(color='blue', icon='flag')).add_to(map_CB)

In [87]:
map_CB.save('./map_CB.html')
CB2.to_csv('./201935256문지환송파커피빈좌표.csv', encoding='CP949', index = False)

In [88]:
import webbrowser

webbrowser.open('./map_CB.html')

True